In [1]:
import psycopg2

In [2]:
# Connecter à PostgreSQL
def create_connection():
    try:
        conn = psycopg2.connect(
            dbname="documents", 
            user="avnadmin", 
            password="AVNS_l7Jb2H8TcLKMke_Jm_j", 
            host="pg-328cb18b-hamdounechaymae-7b98.h.aivencloud.com", 
            port="10809"
        )
        return conn
    except Exception as e:
        print(f"Erreur de connexion à la base de données: {e}")
        return None

In [ ]:
def create_tables():
    commands = (
        """
        CREATE TABLE doc_list (
            id SERIAL PRIMARY KEY,
            title VARCHAR(255) NOT NULL
        )
        """,
        """
        CREATE TABLE term_dict (
            termID SERIAL PRIMARY KEY,
            term VARCHAR(255) NOT NULL
        )
        """,
        """
        CREATE TABLE inverted_index (
            termID INT,
            docID INT,
            freq INT,
            tf_idf FLOAT,
            positions TEXT,
            FOREIGN KEY (termID) REFERENCES term_dict(termID),
            FOREIGN KEY (docID) REFERENCES doc_list(id)
        )
        """
    )
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            for command in commands:
                cur.execute(command)
            cur.close()
            conn.commit()
        except Exception as e:
            print(f"Erreur lors de la création des tables: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")
        
create_tables()

In [ ]:
def check_tables():
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            cur.execute("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public'
            """)
            tables = cur.fetchall()
            print("Tables in the database:")
            for table in tables:
                print(table[0])
            cur.close()
        except Exception as e:
            print(f"Erreur lors de la vérification des tables: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

# Appeler la fonction pour vérifier les tables
check_tables()

In [ ]:
def insert_data_from_csv(table_name, csv_file_path):
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            with open(csv_file_path, 'r') as f:
                next(f)  # Skip the header row
                cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER", f)
            conn.commit()
            cur.close()
            print(f"Data inserted into {table_name} from {csv_file_path}")
        except Exception as e:
            print(f"Erreur lors de l'insertion des données: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

In [ ]:
# def insert_into_doc_list(id, title):
#     conn = create_connection()
#     if conn is not None:
#         try:
#             cur = conn.cursor()
#             cur.execute("INSERT INTO doc_list (id, title) VALUES (%s, %s)", (id, title))
#             conn.commit()
#             cur.close()
#             print(f"Inserted into doc_list: ({id}, {title})")
#         except Exception as e:
#             print(f"Erreur lors de l'insertion dans doc_list: {e}")
#         finally:
#             conn.close()
#     else:
#         print("Erreur de connexion à la base de données.")

# def insert_into_term_dict(termID, term):
#     conn = create_connection()
#     if conn is not None:
#         try:
#             cur = conn.cursor()
#             cur.execute("INSERT INTO term_dict (termID, term) VALUES (%s, %s)", (termID, term))
#             conn.commit()
#             cur.close()
#             print(f"Inserted into term_dict: ({termID}, {term})")
#         except Exception as e:
#             print(f"Erreur lors de l'insertion dans term_dict: {e}")
#         finally:
#             conn.close()
#     else:
#         print("Erreur de connexion à la base de données.")

# # Insert data into doc_list
# insert_into_doc_list(0, "Predisposing factors for incomplete spontaneous recovery after Parsonage-Turner Syndrome.")

# # Insert data into term_dict
# insert_into_term_dict(0, "2000")

In [ ]:
insert_data_from_csv('doc_list', './doc_list.csv')
insert_data_from_csv('term_dict', './term_dict.csv')
insert_data_from_csv('inverted_index', './inverted_index.csv')

In [ ]:
def check_table_contents(table_name):
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            cur.execute(f"SELECT * FROM {table_name}")
            rows = cur.fetchall()
            print(f"Contents of {table_name}:")
            for row in rows:
                print(row)
            cur.close()
        except Exception as e:
            print(f"Erreur lors de la vérification des contenus de la table {table_name}: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

# Example usage
check_table_contents('doc_list')
check_table_contents('term_dict')
check_table_contents('inverted_index')

In [9]:
def add_columns_to_table():
    # Se connecter à la base de données
    conn = create_connection()
    
    if conn is not None:
        cursor = conn.cursor()
        
        try:
            # Requête SQL pour ajouter les champs pdf_file et pdf_excerpt
            alter_table_query = """
            ALTER TABLE doc_list
            ADD COLUMN pdf_file VARCHAR(255) NULL,
            ADD COLUMN pdf_excerpt TEXT NULL;
            """
            
            cursor.execute(alter_table_query)
            conn.commit()  # Appliquer la modification
            
            print("Les colonnes ont été ajoutées avec succès !")
        
        except Exception as e:
            print(f"Erreur lors de l'ajout des colonnes: {e}")
        finally:
            cursor.close()
            conn.close()

# Appeler la fonction pour ajouter les colonnes
add_columns_to_table()

Les colonnes ont été ajoutées avec succès !


In [4]:
def update_pdf_excerpt(title, pdf_excerpt):
    # Se connecter à la base de données
    conn = create_connection()
    
    if conn is not None:
        cursor = conn.cursor()
        
        try:
            # Requête SQL pour mettre à jour l'extrait de texte
            update_query = """
            UPDATE doc_list 
            SET pdf_excerpt = %s
            WHERE title = %s;
            """
            
            # Exécution de la requête
            cursor.execute(update_query, (pdf_excerpt, title))
            conn.commit()  # Appliquer la mise à jour
            
            print(f"Extrait du document '{title}' mis à jour avec succès !")
        
        except Exception as e:
            print(f"Erreur lors de la mise à jour du document '{title}': {e}")
        finally:
            cursor.close()
            conn.close()

In [5]:
def document_exists(title):
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT 1 FROM doc_list WHERE title = %s LIMIT 1;", (title,))
            result = cursor.fetchone()
            return result is not None
        except Exception as e:
            print(f"Erreur lors de la vérification du document '{title}': {e}")
            return False
        finally:
            cursor.close()
            conn.close()

In [6]:
# Fonction pour lire et extraire une portion des fichiers texte
import os 

def process_txt_files(data_folder):
    # Vérifier si le dossier existe
    if not os.path.exists(data_folder):
        print(f"Le dossier {data_folder} n'existe pas.")
        return
    
    # Parcourir les fichiers dans le dossier Data/
    for filename in os.listdir(data_folder):
        # Vérifier si le fichier est un fichier texte
        if filename.endswith(".txt"):
            # Construire le chemin du fichier texte
            txt_file_path = os.path.join(data_folder, filename)
            
            # Extraire le titre (nom sans extension)
            title = filename.replace(".txt", "")
            
            # Vérifier si un document avec ce titre existe déjà dans la base de données
            if document_exists(title):
                # Lire les premiers 500 caractères du fichier texte
                with open(txt_file_path, "r", encoding="utf-8") as file:
                    # Extraire les premiers 500 caractères (ajustez si nécessaire)
                    pdf_excerpt = file.read(500)
                
                # Mettre à jour l'extrait dans la base de données
                update_pdf_excerpt(title, pdf_excerpt)
            else:
                print(f"Le document avec le titre '{title}' n'existe pas dans la base de données.")
        else:
            print(f"Le fichier {filename} n'est pas un fichier texte.")

In [7]:
process_txt_files("../Data/")

Extrait du document 'Predisposing factors for incomplete spontaneous recovery after Parsonage-Turner Syndrome.' mis à jour avec succès !
Extrait du document 'Gastrointestinal LCH a rare manifestation of Langerhans cell histiocytosis' mis à jour avec succès !
Extrait du document 'A rare case report of omental synovial sarcoma complicated hemoperitoneum and literature review' mis à jour avec succès !
Extrait du document 'Chronic brucellosis with sacroiliitis A case report' mis à jour avec succès !
Extrait du document 'Unilateral adrenal metastases as an unusual primary presentation of hepatocellular carcinoma a case report and literature review' mis à jour avec succès !
Extrait du document 'Repair of an iatrogenic lacrimal fistula a case report' mis à jour avec succès !
Extrait du document 'Cutaneous Metastasis of Breast Carcinoma in the Distal Phalanx of the Left Little Finger A Case Report' mis à jour avec succès !
Extrait du document 'Spontaneous cerebrospinal fluid rhinorrhoea a case

In [10]:
import pandas as pd
def get_table_data():
    conn = create_connection()
    if conn is not None:
        cursor = conn.cursor()
        
        try:
            # Requête SQL pour sélectionner toutes les données de la table
            cursor.execute("SELECT * FROM doc_list LIMIT 10;")  # Vous pouvez ajuster la limite ou la requête selon vos besoins
            rows = cursor.fetchall()  # Récupérer toutes les lignes
            
            # Créer un DataFrame pandas pour afficher proprement les résultats
            df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])  # Ajouter les noms de colonnes
            return df
        
        except Exception as e:
            print(f"Erreur lors de la récupération des données: {e}")
            return None
        finally:
            cursor.close()
            conn.close()

# Récupérer les données et les afficher sous forme de DataFrame
data = get_table_data()
if data is not None:
    display(data)

,id,title,pdf_file,pdf_excerpt
0,37,Musculoskeletal manifestations of lower-extrem...,documents/pdfs/Musculoskeletal manifestations ...,This work is distributed under Journal of Bone...
1,30,Genome-wide meta-analysis of myasthenia gravis...,documents/pdfs/Genome-wide meta-analysis of my...,Genome-wide meta-analysis of myasthenia gravis...
2,11,Uncommon site of Brucella endocarditis in a do...,documents/pdfs/Uncommon site of Brucella endoc...,Uncommon site of Brucella endocarditis in a do...
3,23,Brain abscess caused by Streptococcus pyogenes...,documents/pdfs/Brain abscess caused by Strepto...,Brain abscess caused by Streptococcus pyogenes...
4,17,Pediatric neurobrucellosis and atypical Guilla...,documents/pdfs/Pediatric neurobrucellosis and ...,﻿JOURNAL of MEDICINE and LIFE JML | CASE REPOR...
5,26,Therapeutic effect of nimustine in a dog with ...,documents/pdfs/Therapeutic effect of nimustine...,Therapeutic effect of nimustine in a dog with ...
6,5,Repair of an iatrogenic lacrimal fistula a cas...,documents/pdfs/Repair of an iatrogenic lacrima...,Repair of an iatrogenic lacrimal fistula: a ca...
7,7,Spontaneous cerebrospinal fluid rhinorrhoea a ...,documents/pdfs/Spontaneous cerebrospinal fluid...,Spontaneous cerebrospinal fluid rhinorrhoea: a...
8,41,An Interesting Case of Weil_s Disease with Dia...,documents/pdfs/An Interesting Case of Weil's D...,An Interesting Case of Weil’s Disease with Dia...
9,33,Holocord syringomyelia caused by tethered cord...,documents/pdfs/Holocord syringomyelia caused b...,﻿Zheng et al BMC Neurology () : BMC Neurology ...
